## 무작위 행동

- 단순히 랜덤하게 움직이는 에이전트

In [4]:
import gym

# 게임 환경 생성
env = gym.make("FrozenLake-v1", render_mode='human', is_slippery=False) 
print("가능한 상태: ", env.observation_space.n)
print("가능한 행동: ", env.action_space.n)
env.reset()

n=30
env.render()	
for i in range(n): 
  # 무작위로 행동 하나 선택
  action = env.action_space.sample()
  # 지정 행동 실행 후, (새로운 상태, 보상, 종료 여부 등 정보)를 반환
  state, reward, done, truncated, info = env.step(action)
  # (헹동, 상태, 보상) -> 다음(행동, 상태, 보상) -> ...
  print(f"({action}, {state}, {reward})", end="->")
  # 게임을 화면에 렌더
  env.render() 
  # 게임이 끝나면(=done) 종료
  if done:
    break 

env.close()

가능한 상태:  16
가능한 행동:  4


2024-12-04 21:22:32.439 python[18860:418708] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-04 21:22:32.439 python[18860:418708] +[IMKInputSession subclass]: chose IMKInputSession_Modern


(2, 1, 0.0)->(3, 1, 0.0)->(2, 2, 0.0)->(3, 2, 0.0)->(1, 6, 0.0)->(3, 2, 0.0)->(2, 3, 0.0)->(1, 7, 0.0)->

## Q-table 활용 및 Q-값에 근거한 행동 선택

In [3]:
import gym
import numpy as np

# 게임 환경 생성
env = gym.make("FrozenLake-v1", is_slippery=False)

# Q-테이블 초기화
states = env.observation_space.n
actions = env.action_space.n
q_table = np.zeros((states, actions))

n = 500
for i in range(n):
    env.reset()
    state = 0
    done = False

    while not done:
        # 최대 Q값이 0보다 큰지 작은지에 따라,
        if np.argmax(q_table[state]) > 0:
            # Q-테이블에서 가장 큰 값 가지는 행동을 선택
            action = np.argmax(q_table[state])
        else:	
            # 무작위로 행동 선택			
            action = env.action_space.sample()
        # 지정 행동 실행 후, (새로운 상태, 보상, 종료 여부 등 정보)를 반환
        new_state, reward, done, truncated, info = env.step(action)
        # 새로 얻은 정보로 Q-테이블 갱신
        q_table[state, action] =  reward + np.max(q_table[new_state])
        # 다음 상태로 상태 전이
        state = new_state
    
    print(f"{i}번째 에피소드 후 Q-table")
    print(q_table)

0번째 에피소드 후 Q-table
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
1번째 에피소드 후 Q-table
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
2번째 에피소드 후 Q-table
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
3번째 에피소드 후 Q-table
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0

## Q-러닝 

In [6]:
import gym
import numpy as np

# 게임 환경 생성
env = gym.make("FrozenLake-v1", is_slippery=False)

# Q-테이블 초기화
states = env.observation_space.n
actions = env.action_space.n
q_table = np.zeros((states, actions)) 

# 초매개변수 설정
discount_factor = 0.9
epsilon = 0.9
epsilon_decay_factor = 0.999
learning_rate = 0.8
num_episodes = 1000

for i in range(num_episodes):
    env.reset()
    state = 0
    epsilon *= epsilon_decay_factor # 감쇠 적용
    done = False

    while not done:
        # 입실론 크기에 따라 탐사(exploration) 혹은 활용(exploitation)
        if np.random.random() < epsilon:    
            # 무작위로 행동 선택
            action = env.action_space.sample()	
        else:	
            # Q-테이블에서 가장 큰 값 가지는 행동을 선택
            action = np.argmax(q_table[state])
        # 지정 행동 실행 후, (새로운 상태, 보상, 종료 여부 등 정보)를 반환
        new_state, reward, done, truncated, info = env.step(action)
        # 새로 얻은 정보로 Q-테이블 갱신 (최종 공식 적용)
        q_table[state, action] += learning_rate * (reward + discount_factor * np.max(q_table[new_state]) - q_table[state, action])
        # 다음 상태로 상태 전이
        state = new_state

    print(f"{i}번째 에피소드 후 Q-table")
    print(q_table)

0번째 에피소드 후 Q-table
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
1번째 에피소드 후 Q-table
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
2번째 에피소드 후 Q-table
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
3번째 에피소드 후 Q-table
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0